Assigment

In [42]:
import pandas as pd

Load data and map

In [43]:
# I substract 1 because the indices in the file are 1-based
with open('./devkit_object/mapping/train_rand.txt', 'r') as f:
    train_rand_indices = [int(x) - 1 for x in f.read().split(',')]

val = pd.read_csv('./ImageSets/val.txt', header=None, names=['index'])
train = pd.read_csv('./ImageSets/train.txt', header=None, names=['index'])
train_mapping = pd.read_csv('./devkit_object/mapping/train_mapping.txt', header=None, names=['date', 'drive', 'frame'], sep=' ')
train_mapping['drive_number'] = train_mapping['drive'].str.extract(r'drive_(\d+)_sync')


train_rand_mapping = train_mapping.iloc[train_rand_indices].reset_index(drop=True)


print("Train index 0")
print(train_rand_mapping.iloc[train.iloc[0].values[0] ])
print("Train index 1")
print(train_rand_mapping.iloc[train.iloc[1].values[0] ])

print("Val index 0")
print(train_rand_mapping.iloc[val.iloc[0].values[0] ])
print("Val index 1")
print(train_rand_mapping.iloc[val.iloc[1].values[0] ])


Train index 0
date                            2011_09_28
drive           2011_09_28_drive_0106_sync
frame                                   48
drive_number                          0106
Name: 0, dtype: object
Train index 1
date                            2011_09_26
drive           2011_09_26_drive_0046_sync
frame                                    6
drive_number                          0046
Name: 3, dtype: object
Val index 0
date                            2011_09_26
drive           2011_09_26_drive_0101_sync
frame                                  528
drive_number                          0101
Name: 1, dtype: object
Val index 1
date                            2011_09_26
drive           2011_09_26_drive_0086_sync
frame                                  108
drive_number                          0086
Name: 2, dtype: object


In [39]:
print("train_mapping")
print(train_mapping.head())
print("train_rand_mapping")
print(train_rand_mapping.head())
print("train")
print(train.head())
print("val")
print(val.head())

train_mapping
         date                       drive  frame drive_number
0  2011_09_26  2011_09_26_drive_0005_sync    109         0005
1  2011_09_26  2011_09_26_drive_0005_sync    125         0005
2  2011_09_26  2011_09_26_drive_0005_sync    127         0005
3  2011_09_26  2011_09_26_drive_0005_sync    130         0005
4  2011_09_26  2011_09_26_drive_0005_sync    133         0005
train_rand_mapping
         date                       drive  frame drive_number
0  2011_09_28  2011_09_28_drive_0106_sync     48         0106
1  2011_09_26  2011_09_26_drive_0101_sync    528         0101
2  2011_09_26  2011_09_26_drive_0086_sync    108         0086
3  2011_09_26  2011_09_26_drive_0046_sync      6         0046
4  2011_09_26  2011_09_26_drive_0101_sync    765         0101
train
   index
0      0
1      3
2      7
3      9
4     10
val
   index
0      0
1      2
2      4
3      5
4      6


In [34]:
# Extract unique date and drive number pairs from train set
train_date_drive_pairs = train_rand_mapping.iloc[train['index'].values][['date', 'drive_number']].drop_duplicates()

print(train_date_drive_pairs.head())
# Extract unique date and drive number pairs from val set
val_date_drive_pairs = train_rand_mapping.iloc[val['index'].values][['date', 'drive_number']].drop_duplicates()
print(val_date_drive_pairs.head())
# Check if any date and drive number pairs are common between train and val sets
common_date_drive_pairs = train_date_drive_pairs.merge(val_date_drive_pairs, on=['date', 'drive_number'])

if common_date_drive_pairs.empty:
    print("The train and val sets contain different video sequences.")
else:
    print("The train and val sets have some common video sequences:")
    print(common_date_drive_pairs)

          date drive_number
0   2011_09_28         0106
3   2011_09_26         0046
7   2011_09_26         0056
10  2011_09_26         0036
11  2011_09_26         0106
         date drive_number
0  2011_09_28         0106
2  2011_09_26         0086
4  2011_09_26         0101
5  2011_09_26         0070
6  2011_09_29         0026
The train and val sets have some common video sequences:
         date drive_number
0  2011_09_28         0106


Create new sets

In [110]:
import random

test = val

# Shuffle the unique date and drive pairs in the train set + reset the index
train_shuffled = train.sample(frac=1, random_state=42).reset_index(drop=True)

# Calculate the split point
size_of_train = 3340
size_of_val = 372

# Create a dictionary to group the train_rand_mapping from train_shuffled by video sequence
grouped_by_video = {}
for i, row in train_shuffled.iterrows():
    date = train_rand_mapping.iloc[row['index']]['date']
    drive_number = train_rand_mapping.iloc[row['index']]['drive_number']
    video_sequence = (date, drive_number)
    if video_sequence not in grouped_by_video:
        grouped_by_video[video_sequence] = []
    grouped_by_video[video_sequence].append(row['index'])

# Shuffle the video sequences
video_sequences = list(grouped_by_video.keys())
random.shuffle(video_sequences)

selected_video_sequences = {}
current_size = 0

for video_sequence in video_sequences:
    indices = grouped_by_video[video_sequence]
    
    if current_size + len(indices) == size_of_val:
        selected_video_sequences[video_sequence] = indices
        current_size += len(indices)
        break
    
    if current_size + len(indices) < size_of_val:
        selected_video_sequences[video_sequence] = indices
        current_size += len(indices)


# Remove selected video sequences from the grouped_by_video dictionary
for video_sequence in selected_video_sequences:
    grouped_by_video.pop(video_sequence)

# Check the total number of frames in selected_video_sequences and grouped_by_video
total_frames = sum([len(x) for x in selected_video_sequences.values()]) + sum([len(x) for x in grouped_by_video.values()])
print("Total number of frames in selected_video_sequences and grouped_by_video: {}".format(total_frames))

# Print the results
print("Number of unique video sequences in val set: {}".format(len(selected_video_sequences)))
print(selected_video_sequences)
print("Number of frames in val set: {}".format(current_size))

print("Number of unique video sequences in train set: {}".format(len(grouped_by_video)))
print(grouped_by_video)
print("Number of frames in train set: {}".format(sum([len(x) for x in grouped_by_video.values()])))

# Create a new train set
train_new = pd.DataFrame(columns=['index'])
for video_sequence in grouped_by_video:
    for index in grouped_by_video[video_sequence]:
        train_new = train_new.append({'index': index}, ignore_index=True)

# Create a new val set
val_new = pd.DataFrame(columns=['index'])
for video_sequence in selected_video_sequences:
    for index in selected_video_sequences[video_sequence]:
        val_new = val_new.append({'index': index}, ignore_index=True)

# Chech if the new train and val sets have different video sequences
train_date_drive_pairs = train_rand_mapping.iloc[train_new['index'].values][['date', 'drive_number']].drop_duplicates()
val_date_drive_pairs = train_rand_mapping.iloc[val_new['index'].values][['date', 'drive_number']].drop_duplicates()
common_date_drive_pairs = train_date_drive_pairs.merge(val_date_drive_pairs, on=['date', 'drive_number'])

if common_date_drive_pairs.empty:
    print("The new train and val sets contain different video sequences.")
else:
    print("The new train and val sets have some common video sequences:")
    print(common_date_drive_pairs)



Total number of frames in selected_video_sequences and grouped_by_video: 3712
Number of unique video sequences in val set: 10
{('2011_09_26', '0087'): [29, 3875, 6947, 901, 895, 710, 570, 934, 584, 2143, 4853, 7009, 3825, 6373, 7438, 444, 3326, 1074, 7062, 1657, 2316, 629, 467, 900, 5789, 552, 2808, 2438, 2921, 6171, 1338, 4316, 6541, 7040, 2447, 3613, 3400, 6101, 2431, 4600, 282, 6607, 6142, 4451, 5003, 2213, 1601, 6392, 6566, 4761, 6230, 1805, 563, 7175, 30, 7441, 7418, 3420, 6934, 3957, 304, 1599, 6193, 4678, 1005, 2498, 1297, 5212, 4302, 5966, 738, 6383, 3243, 5463, 6091, 1949, 824, 4890, 1079, 5896, 730, 1379, 6986, 6466, 6359, 534, 954, 2265, 3445, 1105, 1724, 2969, 2026, 299, 4495, 240], ('2011_09_28', '0068'): [5808, 1160, 7362, 5005, 3059, 2342], ('2011_09_26', '0011'): [4796, 632, 3861, 4952, 2084, 6564, 3108, 763, 2096, 5215, 2973, 2802, 5517, 2913, 4945, 1950, 1184, 5043, 154, 3641, 4499, 2322, 2571, 6072, 830, 3868, 5232, 6109, 127, 5991, 3556, 2466, 4964, 1523, 902, 5792,